In [9]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [11]:
transform = transforms.ToTensor()

train_data = torchvision.datasets.MNIST(root='./', train=True, download=True, transform=transform)
test_data = torchvision.datasets.MNIST(root='./', train=False, download=True, transform=transform)

In [12]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Split: train
    Root Location: ./
    Transforms (if any): ToTensor()
    Target Transforms (if any): None

In [13]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Split: test
    Root Location: ./
    Transforms (if any): ToTensor()
    Target Transforms (if any): None

In [14]:
# Hyper-parameters 
input_size = 784 # 28x28
hidden_size = 500 
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [15]:
train_loader = torch.utils.data.DataLoader(dataset=train_data, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_data, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [16]:
examples = iter(train_loader)
example_data, example_targets = examples.next()

for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(example_data[i][0], cmap='gray')
plt.show()

AttributeError: '_SingleProcessDataLoaderIter' object has no attribute 'next'

In [17]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)  

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        # no activation and no softmax at the end
        return out

In [18]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [19]:
#loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [20]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # origin shape: [100, 1, 28, 28]
        # resized: [100, 784]
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/2], Step [100/600], Loss: 0.3113
Epoch [1/2], Step [200/600], Loss: 0.2953
Epoch [1/2], Step [300/600], Loss: 0.2119
Epoch [1/2], Step [400/600], Loss: 0.1848
Epoch [1/2], Step [500/600], Loss: 0.1994
Epoch [1/2], Step [600/600], Loss: 0.1484
Epoch [2/2], Step [100/600], Loss: 0.1329
Epoch [2/2], Step [200/600], Loss: 0.0916
Epoch [2/2], Step [300/600], Loss: 0.1354
Epoch [2/2], Step [400/600], Loss: 0.1244
Epoch [2/2], Step [500/600], Loss: 0.0970
Epoch [2/2], Step [600/600], Loss: 0.0640


In [22]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')

Accuracy of the network on the 10000 test images: 97.09 %
